In [ ]:
import ipywidgets as widgets # provides interactive functionality
from IPython.display import HTML, display, clear_output # display and html functionality
import pandas as pd
import csv # csv import functionality
import warnings # needed for suppression of unnecessary warnings
import base64 # functionality for encoding binary data to ASCII characters and decoding back to binary data
import fit_models as fits
import plot_fits as plot_fits
import hospital_needs as needs

warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'svg' # configure to display scalable vector graphics

<img src="RUSH_full_color.jpg" style="float:left; margin:0px 0px 0px 0px" markdown="1" width="400"/>
<font size="5" color='green'>Center for Quality, Safety & Value Analytics</font>

[Click for instructions on using this calculator](https://webalyticos.home.blog/2020/03/24/covd19forecast/)  

In [ ]:
testing = 0

if testing == 1:
    seir_fits_df = pd.read_csv('SEIR-SD_States_Update.txt', sep='\t')
else:
    try:
        seir_fits_df = pd.read_csv('https://raw.githubusercontent.com/Rush-Quality-Analytics/SupplyDemand/master/notebooks/SEIR-SD_States_Update.txt', sep='\t')
    except:
        seir_fits_df = pd.read_csv('SEIR-SD_States_Update.txt', sep='\t')

if testing == 1:
    statepops = pd.read_csv('StatePops.csv')
else:
    try: 
        path = 'https://raw.githubusercontent.com/Rush-Quality-Analytics/SupplyDemand/master/notebooks/StatePops.csv'
        statepops = pd.read_csv('path')

    except:
        statepops = pd.read_csv('StatePops.csv')

if testing == 1:
    ap_df = pd.read_csv('COVID-CASES-DF.txt', sep='\t') 
    ap_df = ap_df[ap_df['Country/Region'] == 'US']
    ap_df = ap_df[ap_df['Province/State'] != 'US']
    ap_df = ap_df[ap_df['Province/State'] != 'American Samoa']
    ap_df = ap_df[ap_df['Province/State'] != 'Northern Mariana Islands']
    ap_df.drop(columns=['Unnamed: 0'], inplace=True)
    
else:
    try: 
        ap_df = pd.read_csv('https://raw.githubusercontent.com/Rush-Quality-Analytics/SupplyDemand/master/notebooks/COVID-CASES-DF.txt', sep='\t') 
        ap_df = ap_df[ap_df['Country/Region'] == 'US']
        ap_df = ap_df[ap_df['Province/State'] != 'US']
        ap_df = ap_df[ap_df['Province/State'] != 'American Samoa']
        ap_df = ap_df[ap_df['Province/State'] != 'Northern Mariana Islands']
        ap_df.drop(columns=['Unnamed: 0'], inplace=True)
    except:
        ap_df = pd.read_csv('COVID-CASES-DF.txt', sep='\t') 
        ap_df = ap_df[ap_df['Country/Region'] == 'US']
        ap_df = ap_df[ap_df['Province/State'] != 'US']
        ap_df = ap_df[ap_df['Province/State'] != 'American Samoa']
        ap_df = ap_df[ap_df['Province/State'] != 'Northern Mariana Islands']
        ap_df.drop(columns=['Unnamed: 0'], inplace=True)



    
global app2    
app0 = fits.App_GetFits.fits(ap_df, statepops, seir_fits_df)
grid0 = widgets.GridspecLayout(1, 1)
grid0[0, 0] = app0.container
app_contents0 = [grid0]
app00 = widgets.VBox(app_contents0, layout=widgets.Layout(align_items='center'))
display(app00)



b1 = widgets.Button(description='Click to update plots and tables. Do this after running a different model and/or choosing a different state or territory.',
    disabled=False, button_style='info', tooltip='Click me', layout={'width': '1000px'}, icon='')
box_layout = widgets.Layout(display='flex',flex_flow='column',align_items='center',width='100%')
box1 = widgets.HBox(children=[b1],layout=box_layout)
display(box1)
out1 = widgets.Output()
app1 = plot_fits.App_PlotFits.generate_plot(app0._model_fits_df)

with out1:    
    grid1 = widgets.GridspecLayout(1, 1)
    grid1[0, 0] = app1.container
    app_contents1 = [grid1]
    app11 = widgets.VBox(app_contents1, layout=widgets.Layout(align_items='center'))
    display(app11)
display(out1)


b2 = widgets.Button(description='Click to display or update downloadable csv files.',
    disabled=False, button_style='info', tooltip='Click me', layout={'width': '1000px'}, icon='')
box_layout = widgets.Layout(display='flex',flex_flow='column',align_items='center',width='100%')
box2 = widgets.HBox(children=[b2],layout=box_layout)

out2 = widgets.Output()
app2 = needs.App_GetNeeds.from_url(app0.ForecastDays, app0._model_fits_df, app0.new_cases)

with out2:    
    grid2 = widgets.GridspecLayout(1, 1)
    grid2[0, 0] = app2.container
    app_contents2 = [grid2]
    app22 = widgets.VBox(app_contents2, layout=widgets.Layout(align_items='center'))
    display(app22)
display(out2)
display(box2)


def on_button_clicked1(b1):
    global app2, app0
    def update_out2(out2):
        global app2, app0
        with out2:
            app2 = needs.App_GetNeeds.from_url(app0.ForecastDays, app0._model_fits_df, app0.new_cases)
            grid2 = widgets.GridspecLayout(1, 1)
            grid2[0, 0] = app2.container
            app_contents2 = [grid2]
            app22 = widgets.VBox(app_contents2, layout=widgets.Layout(align_items='center'))
            clear_output(wait=True)
            display(app22)
            
        
    with out1:
        app1 = plot_fits.App_PlotFits.generate_plot(app0._model_fits_df)
        grid1 = widgets.GridspecLayout(1, 1)
        grid1[0, 0] = app1.container
        app_contents1 = [grid1]
        app11 = widgets.VBox(app_contents1, layout=widgets.Layout(align_items='center'))
        clear_output(wait=True)
        display(app11)  
        update_out2(out2)
    
b1.on_click(on_button_clicked1)



def on_button_clicked2(b2):
    
    with out2:
        
        grid2 = widgets.GridspecLayout(1, 1)
        grid2[0, 0] = app2.container
        app_contents2 = [grid2]
        app22 = widgets.VBox(app_contents2, layout=widgets.Layout(align_items='center'))
        clear_output(wait=True)
        display(app22)
        
        model = app0.model
        df = app2.Forecasted_cases_df_for_download
        title = "Download CSV file of forecasted COVID-19 cases for " + model + " model"
        filename = "Forecasted_COVID19_Cases.csv"
        csv = df.to_csv()
        b64 = base64.b64encode(csv.encode())
        payload = b64.decode()
        html = '<a download="{filename}" href="data:text/csv;base64,{payload}" style="left; margin:0px 0px 0px 0px" size="4" target="_blank">{title}</a>'
        html = html.format(payload=payload,title=title,filename=filename)
        display(HTML(html))
        
        
        df = app2.Forecasted_patient_census_df_for_download
        title = "Download CSV file of forecasted patient census for " + model + " model" 
        filename = "Forecasted_COVID19_Patient_Census.csv"
        csv = df.to_csv()
        b64 = base64.b64encode(csv.encode())
        payload = b64.decode()
        html = '<a download="{filename}" href="data:text/csv;base64,{payload}" style="left; margin:0px 0px 0px 0px" size="4" target="_blank">{title}</a>'
        html = html.format(payload=payload,title=title,filename=filename)
        display(HTML(html))
        
        
        df = app2.Forecasted_ppe_needs_df_for_download
        title = "Download CSV file of forecasted PPE needs for " + model + " model"
        filename = "Forecasted_COVID19_PPE_Needs.csv"
        csv = df.to_csv()
        b64 = base64.b64encode(csv.encode())
        payload = b64.decode()
        html = '<a download="{filename}" href="data:text/csv;base64,{payload}" style="left; margin:0px 0px 0px 0px" size="4" target="_blank">{title}</a>'
        html = html.format(payload=payload,title=title,filename=filename)
        display(HTML(html))
        
        
b2.on_click(on_button_clicked2)


<a href="https://github.com/Rush-Quality-Analytics/SupplyDemand/" target="_blank"> 
<img width="220" height="120" border="0" align="left"  src="github.png"/> 
</a>


-----
<font size="4" color='green'>Model Descriptions</font>
<body style="margin: 0px 0px 0px 0px; background-color: rgb(255, 255, 255); color: rgb(0, 0, 0);">
  <p style="text-align: justify;">Exponential:&nbsp;<span style="color: rgb(169, 169, 169);">The exponential model has been widely successful in capturing the increase in COVID-19 cases during the most rapid and difficult-to-mitigate phases. The exponential model takes a simple form and essentially captures the effect of repeated doubling over time (1, 2, 4, 8, …). In our modeling, predicted values of the exponential were determined by linear regression conducted on log(numbers of cases).</span></p>
  <p style="text-align: justify;">Quadratic 2nd order polynomial: <span style="color: rgb(169, 169, 169);">A 2nd order polynomial (y ~ x2 + x) captures quadratic growth and is the expected outcome when the growth rate changes and when that rate of change is constant. The rate of increase in this model is initially faster than that of the exponential model. We obtained predicted values from this model using numerical optimization of parameters and curve fitting.</span></p>
  <p style="text-align: justify;">Logistic: <span style="color: rgb(169, 169, 169);">When exponential growth slows and tapers-off, the growth curve often becomes logistic, that is, “S” shaped. The rate of increase in this model is initially exponential but slows as an upper limit is approached. We obtained predicted values from this model using numerical optimization of parameters and curve fitting.</span></p>
  <p style="text-align: justify;">SEIR-SD: <span style="color: rgb(169, 169, 169);">An epidemiological model that attempts to predict the changes in numbers of people who are susceptible to COVID-19 infection, who have been exposed to COVID-19, who have been infected and are symptomatic, and who have recovered. This SEIR model uses the total population size of your chosen location (source: US Census 2010-2019). It attempts to infer the date of the first COVID-19 case in your chosen location, the average incubation period of COVID-19, average infectious period, and the initial reproductive number. The 'SD' in the model pertains to the inclusion of social distancing. The model assumes that social distancing naturally becomes more prevalent as the percent of the population infected with COVID-19 increases. The model also assumes that testing for COVID-19 was initially low but increased during the weeks following the first reported cases in the US.</span></p>
  <p style="text-align: justify;">Coefficients of determination (r-square values): <span style="color: rgb(169, 169, 169);">These pertain to the relationship of observed values vs. predicted values and so, reveal the percent of variation in the observed values explained by the predicted values.</span></p>
  
  <font size="4" color='green'>Developer</font>
  <p style="text-align: justify;">Ken Locey, PhD, Data Science Analyst</p>
  <font size="4" color='green'>Site Architect and Administrator</font>
  <p style="text-align: justify;">Jawad Khan, AVP, Advanced Analytics &amp; Knowledge Management</p>
  <font size="4" color='green'>Center for Quality, Safety &amp; Value Analytics Leadership</font>
  <p style="text-align: justify;">Thomas A. Webb, MBA, Associate Vice President</p>
  <p style="text-align: justify;">Bala N. Hota, MD, MPH, Vice President, Chief Analytics Officer</p>
  <p>
  <br>
  </p>
</body>